# Sistemas Inteligentes

## Formulação de Problemas de Satisfação de Restrições


## Conteúdos

* A classe CSP
    * definição de variáveis, domínios, vizinhos e restrições
* Problema das três variáveis todas diferentes
* Problema dos jogadores de Golfe
* Exercícios

### Introdução

Nesta aula vamos formular e resolver problemas de satisfação de restrições - **Constraint Satisfaction Problems (CSP)**.
O contexto é o analisado nas aulas T e TP em que um CSP é caracterizado por 3 componentes:
- **Variáveis** - para as quais se quer encontrar uma afectação (*assigment*) completa e consistente (solução); podem ser discretas ou contínuas.
- **Domínios** - indicam os valores possíveis das variáveis; podem ser finitos ou infinitos.
- **Restrições** - especificam combinações possíveis de valores para subconjuntos de variáveis; podem ser: 
    1. unárias, por exemplo `<(X1): X1=5>`, 
    2. binárias, por exemplo `<(X1,X2): X1=X2>`, ou de 
    3. ordem superior, por exemplo `<(X1,X2,X3), alldiff(X1,X2,X3)>`.

Neste contexto, **em CSP**:

* **Estado** - é um conjunto de variáveis com valores nos seus domínios (afectação/assignment); uma afectação pode ser (1) completa, se todas as variáveis têm valor, ou parcial, caso contrário; e (2) consistente, se  satisfaz todas as restrições, ou inconsistente, caso contrário. 
    
* **Inferência e Procura** - é efectuada usando propagação de restrições e, quando ainda há diversas soluções possíveis, algoritmos de procura que usam heurísticas genéricas em vez de heurística específica do problema, como acontecia na resolução dos Problemas de Espaços de Estados (PEE).

* **Solução** - é uma afectação completa e consistente.

**A resolução de um CSP tem 2 passos**:

**Passo 1)** Formulação do problema (definir variáveis, domínios, restrições)

**Passo 2)** Resolução do problema (propagar restrições e, caso necessário, usar algoritmo de procura)

Nesta aula iremos apenas dedicar-nos à formulação.

In [1]:
from csp import *

### Módulo *CSP.py* - Breve Explicação

Este módulo foi construído com base no módulo **CSP.py** disponível no [repositório habitual](https://github.com/aimacode/aima-python). 
No essencial, é disponibilizado o seguinte:
* Classe **CSP**, que serve de base à definição dos problemas CSP;
* Funções que implementam os métodos de resolução de problemas CSP, que trataremos apenas na aula seguinte.

Este módulo tem que ser importado (como ilustrado acima), mas **não deverá alterá-lo**.

**Note que este módulo lida apenas com restrições binárias!** As restrições unárias terão de ser eliminadas de modo a verificar a consistência das variáveis. Os domínios reflectem essas restrições. 

### Criação de um CSP usando a classe **CSP**

Esta classe permite definir um CSP assumindo que os domínios das variáveis são finitos. Para definir um CSP concreto é necessário definir as variáveis, os domínios, o grafo de restrições e as restrições que serão depois usadas para criar um objecto da classe CSP.

Neste contexto, o construtor **`def __init__(self, variables, domains, neighbors, constraints)`** tem os seguintes argumentos:
  
* **`variables`** : Lista de variáveis (strings ou inteiros).

* **`domains`** : Dicionário com elementos do tipo `{var:[valor, ...]}`.

* **`neighbors`**: Dicionário com elementos do tipo `{var:[var,...]}` em que cada variável var (chave) tem como valor uma lista das variáveis com as quais tem restrições (vizinhos no grafo de restrições), que define o grafo de restrições.

* **`constraints`** : Função do tipo `f(A, a, B, b)` que devolve `True` se os vizinhos `A` e `B` satisfazem a restrição quando têm valores `A=a` e `B=b`. 

### Problema das três variáveis A, B e C tal que todas são diferentes
Vamos formmular o problema de encontrar valores para as 3 variáveis A, B e C tais que sejam todas diferentes, quando o domínio é {1,2,3}
Para isso vamos definir as variáveis, os domínios, a vizinhança entre as variáveis (as que estão ligadas por restrições binárias) e a função que verifica se os valores de duas variáveis vizinhas são consistentes, não violando a respectivas restrições.

#### Definição das variáveis
Temos 3 variáveis A, B e C.

Vamos ter a lista
``` python
    variaveis_ABCdifs = ['A', 'B', 'C']
```

#### Definição dos domínios
Os domínios vão ser listas de números. Neste caso referem-se à lista formada por 1, 2 e 3.

O domínio vai ser o mesmo para todas as variáveis.

``` python
    dominios_ABCdifs = {}
    for v in variaveis_ABCdifs :
        dominios_ABCdifs[v] = [1,2,3]  
```

#### Definir Vizinhos
Vamos criar o grafo de restrições com os arcos seguintes:
    
    A - B, A - C, B - C
    
    Para isso usaremos a função 
```python
    parse_neighbors('A : B C; B: C; C : ')
```

É necessário perceber o que faz o `parse_neighbors`. Converte uma string da forma 'X: Y Z; Y: Z' num dicionário como este:
```python
    {'Y': ['X', 'Z'], 'X': ['Y', 'Z'], 'Z': ['X', 'Y']}
```
Declarando 'X: Y' não precisamos de declarar 'Y: X'.
A função `parse_neighbors()` irá devolver um `defaultdict` que não é mais do que um dicionário com uma função factory para gerar o valor por defeito quando se pede o valor de uma chave que ainda não existe. Neste caso, gera-se por omissão uma lista vazia. 

#### Definir a função  que verifica a satisfação das restrições entre duas variáveis afectadas
Precisamos de uma função que verifique se duas variáveis X, e Y vizinhas satisfazem todas as restrições binárias em que estão envolvidas, quando afetadas respetivamente aos valores a e b.

Essa função neste caso é a mesma para todos as variáveis vizinhas, eles têm de ter valores diferentes entre si. Vamos usar a função ***different_values_constraint()*** definida em `csp.py`

```python
    def different_values_constraint(A, a, B, b):
        """A constraint saying two neighboring variables must differ in value."""
        return a != b
```

Vamos então criar o CSP desejado utilizando uma função.

In [3]:
def CSP_ABCdifs():
    """
    Retorna um objecto da classe CSP inicializado com as variáveis, os domínios,
    os vizinhos e restrições do problema de em que todos são diferentes entre si.
    """
    
    #Definir Variáveis
    variaveis_ABCdifs = 'A B C'.split()
        
    # Definir Domínios
    # Devolve um dicionario com os domínios com as variáveis do problema dos Golfers.
    # Deveria ser {1,2,3} para todas as variáveis, mas dado que a implementação 
    # em CSP assume que as restrições são binárias.     
    dominios_ABCdifs = {}
    for v in variaveis_ABCdifs :
        dominios_ABCdifs[v] = [1,2,3]
    
    #Definir Vizinhos
    #Cria o grafo de restrições com os arcos seguintes:
    #A : B
    #B : C
    
    vizinhos_ABCdifs = parse_neighbors('A: B C; B: C')

        
    return CSP(variaveis_ABCdifs, dominios_ABCdifs, vizinhos_ABCdifs, different_values_constraint)

p = CSP_ABCdifs()

Podemos agora ler os atributos do objecto criado:

In [4]:
print("Variáveis = ", p.variables)
print("Domínios = ", p.domains)
print("Variáveis = ", p.neighbors)
print("Restrições",p.constraints)

Variáveis =  ['A', 'B', 'C']
Domínios =  {'A': [1, 2, 3], 'B': [1, 2, 3], 'C': [1, 2, 3]}
Variáveis =  defaultdict(<class 'list'>, {'A': ['B', 'C'], 'B': ['A', 'C'], 'C': ['A', 'B']})
Restrições <function different_values_constraint at 0x7f64bc6315a0>


Invoquemos a restrição sobre duas variáveis:

In [5]:
p.constraints('A',2,'B',3)

True

In [6]:
p.constraints('A',2,'B',2)

False

### Problema dos Jogadores de Golfe (Golfers)

Vamos agora formular o problema dos Golfers: Quatro jogadores de golfe (Bob, Fred, Joe, Tom) estão alinhados, da esquerda para a direita em posições numeradas de 1 a 4. Todos têm calças de cor diferente. Tendo em conta a informação a seguir formule o problema de modo a determinar a ordem pela qual os golfistas estão alinhados e a cor das respectivas calças.

    Um tem calças vermelhas;
    O que está à direita do Fred tem calças azuis;
    Joe é o segundo no alinhamento;
    Bob tem calças pretas;
    Tom não está na posição 4, e não é ele que tem as calças cor de laranja.

#### Formulação do CSP Golfers

Por forma formular o problema implementamos a **função `CSP_golfers`** que devolve um objecto da classe `CSP` inicializado com as variáveis, os domínios, os vizinhos e as restrições do problema dos Golfers.

Antes de criar o objecto da classe `CSP` temos que definir:

* **`variaveis_golfers`**
* **`dominios_golfers`**
* **`vizinhos_golfers`**
* **`restricoes_golfers`**

de forma a passar estes argumentos ao construtor da classe `CSP`. 

Repare ainda que `restricoes_golfers` é uma função que avalia as restrições binárias do problema dos Golfers.

Vamos formular o problema dos Golfers considerando que temos 8 variáveis: os nomes dos golfers e as cores das calças e que os domínios serão as posições entre 1 e 4. Assim, a afectação {Bob = 1, Azuis = 1} significa que o Bob está na posição 1 e quem veste calças azuis também, i.e. o Bob veste calças azuis e está na primeira posição.

Vamos de seguida descrever a função `CSP_golfers` com algum detalhe, nos seus 4 componentes.

#### Definição das variáveis
Temos 4 variáveis que se referem aos jogadores de Golf e também temos variáveis que se referem a quem tem uma determinada côr de calças.

Vamos criar duas listas com os dois conjuntos de variáveis. Podíamos ter explicitado as listas mas resolvemos usar o método split().

``` python
    golfers = 'Bob Fred Joe Tom'.split()
    cores_calcas = 'Azuis Laranja Pretas Vermelhas'.split()
```
Vamos criar uma lista com todas as variáveis:
``` python
variaveis_golfers = golfers + cores_calcas
```

#### Definição dos domínios
Os domínios vão ser listas de números. Neste caso referem-se às posições, que variam de 1 a 4.
Essas listas são guardadas num dicionário, em que as chaves são as variáveis e os valores os domínios respectivos.

O domínio deveria ser o mesmo para todas as variáveis, {1,2,3,4}, mas dado que a implementação em CSP assume que as restrições são binárias, as restrições unárias foram eliminadas verificando a consistência das variáveis e os domínios das variáveis 'Joe' e 'Tom' já reflectem  essas restrições.

``` python
    dominios_golfers = {}
    for v in variaveis_golfers :
        dominios_golfers[v] = [1,2,3,4]
    dominios_golfers['Joe'] = [2]
    dominios_golfers['Tom'] = [1,2,3]  
```

#### Definir Vizinhos
Vamos criar o grafo de restrições com os arcos seguintes:
    
    Bob : Fred Joe Tom Pretas
    Fred: Joe Tom Azuis
    Joe : Tom
    Tom : Laranja
    Azuis : Laranja Pretas Vermelhas
    Laranja : Pretas Vermelhas
    Pretas : Vermelhas

```python

vizinhos_golfers = parse_neighbors('Bob : Fred Joe Tom Pretas; \
        Fred: Joe Tom Azuis; Joe : Tom ; Tom : Laranja; \
        Azuis : Laranja Pretas Vermelhas; Laranja : Pretas Vermelhas; \
        Pretas : Vermelhas')
```

#### Definir a função  que verifica a satisfação das restrições entre duas variáveis afectadas
Precisamos de uma função que verifique se duas variáveis X, e Y satisfazem todas as restrições binárias em que estão envolvidas, quando afetadas respetivamente aos valores a e b.

Essa função vai criar todas as restrições e depois verificar a satisfabilidade de todas as que envolvam o par de variáveis. Neste caso, dos Golfers, só teremos uma restrição por par.


Cada restrição será uma função:

Eis as binárias:

        Retrições Binárias:
        R_B1) Fred = Azuis - 1  [equivalente a Azul = Fred + 1]
        R_B2) Bob = Pretas
        R_B3) Tom <> Laranja

As não binárias (globais) terão de ser transformadas em binárias envolvendo apenas o <>.

        R_G1) allDiff(golfers)
        R_G2) allDiff(cores_calcas)


Todas as condições nas restrições, serão implementadas em python como funções

``` python
        def menos1(x,y):
            return (x == y - 1)
        
        def mais1(x,y):
            return (x == y + 1)
        
        def iguais(x,y):
            return x == y
        
        def diferentes(x,y):
            return x != y
```

Vamos guardar as restrições binárias num dicionário em que cada chave é um par de variáveis e o valor é uma restrição (função). Note que vamos incluir as condições para o par (X,Y) e o seu simétrico. É necessário fazer isto porque, ao contrário da vizinhança, que é uma relação cumutativa, algumas restrições, como por exemplo "menos1" não são cumutativas.

```python
# restrições binárias (necessário incluir (X,Y) e (Y,X))
restricoes = { ('Fred','Azuis') : menos1, 
               ('Azuis','Fred') : mais1, 
               ('Bob','Pretas') : iguais, 
               ('Pretas','Bob') : iguais,
               ('Tom','Laranja') : diferentes, 
               ('Laranja','Tom') : differentes,
               ('Laranja','Pretas') : diferentes, 
               ('Pretas','Laranja') : diferentes}
```

Iremos adicionar de modo automático a esse dicionário as restrições binárias que resultam das restrições globais:
```python
# all-different em cada categoria (golfers e cores)
if X in golfers and Y in golfers or \
            X in cores_calcas and Y in cores_calcas :
    restricoes[(X,Y)] = diferentes
```  

No final da geração das restrições, é necessário verificar se X=a e Y=b satisfazem a restrição, que depende da ordem do par. Se não houver uma restrição entre essas duas variáveis então o resultado é True.

Assim, a função `restricoes_golfers(X, a, Y, b)`, depois de se terem definido as condições e o dicionário com as restrições, irá devolver:
        
```python
if (X,Y) in restricoes :
    return restricoes[(X,Y)](a,b)
else: # Se não há restrição
    True
```


Depois da explicação em detalhe da geração dos componentes necessários para gerar uma instância da classe CSP para o problemas dos Golfers, eis o código completo. 

In [8]:
# Funções binárias usadas nas restrições
def menos1(x,y):
    """x é igual a y-1"""
    return (x == y - 1)
        
def mais1(x,y):
    """x é igual a y+1"""
    return (x == y + 1)
        
def iguais(x,y):
    """x é igual a y"""
    return x == y
        
def diferentes(x,y):
    """x diferente de y"""
    return x != y


def CSP_golfers ():
    """
    Retorna um objecto da classe CSP inicializado com as variáveis, os domínios,
    os vizinhos e restrições do problema dos Golfers
    """
    
    #Definir Variáveis
    #golfers - posicao dos golfers - {1,2,3,4}
    #cores_calcas - posicao das cores - {1,2,3,4}
    golfers = 'Bob Fred Joe Tom'.split()
    cores_calcas = 'Azuis Laranja Pretas Vermelhas'.split()
    variaveis_golfers = golfers + cores_calcas
    
    # Definir Domínios
    # Devolve um dicionario com os domínios com as variáveis do problema dos Golfers.
    # Deveria ser {1,2,3,4} para todas as variáveis, mas dado que a implementação 
    # em CSP assume que as restrições são binárias, as restrições unárias foram 
    # eliminadas verificando a consistencia das variáveis e os domínios das variáveis 'Joe' e 'Tom' já reflectem 
    # essas restrições.      
    dominios_golfers = {}
    for v in variaveis_golfers :
        dominios_golfers[v] = [1,2,3,4]
    dominios_golfers['Joe'] = [2]
    dominios_golfers['Tom'] = [1,2,3]  
 
    #Definir Vizinhos
    #Cria o grafo de restrições com os arcos seguintes:
    #Bob : Fred Joe Tom Pretas
    #Fred: Joe Tom Azuis
    #Joe : Tom
    #Tom : Laranja
    #Azuis : Laranja Pretas Vermelhas
    #Laranja : Pretas Vermelhas
    #Pretas : Vermelhas
    
    vizinhos_golfers = parse_neighbors('Bob: Fred Joe Tom Pretas; \
        Fred: Joe Tom Azuis; Joe: Tom; Tom: Laranja; \
        Azuis: Laranja Pretas Vermelhas; Laranja: Pretas Vermelhas; \
        Pretas: Vermelhas')

    #Definir função que verifica restrições binárias
    def restricoes_golfers(X, a, Y, b) :
        """
        A implementação em CSP assume que as restrições são binárias.
        Retorna True se (X=a,Y=b) satisfaz as restrições entre X e Y.
        
        Restrições unárias:
        R_U1) Joe = 2 obriga a que domínio Joe = {2}
        R_U2) Tom <> 4 obriga a que domínio Tom = {1,2,3}
        
        Retrições Binárias:
        R_B1) Fred = Azuis - 1  [equivalente a Azul = Fred + 1]
        R_B2) Bob = Pretas
        R_B3) Tom <> Laranja
        
        Restrições Globais:
        R_G1) allDiff(golfers)
        R_G2) allDiff(cores_calcas)
        """
        
        # restrições binárias (necessário incluir (X,Y) e (Y,X))
        restricoes = { ('Fred','Azuis') : menos1, ('Azuis','Fred') : mais1, 
                       ('Bob','Pretas') : iguais, ('Pretas','Bob') : iguais,
                       ('Tom','Laranja') : diferentes, ('Laranja','Tom') : diferentes,
                       ('Laranja','Pretas') : diferentes, ('Pretas','Laranja') : diferentes}

        
        # all-different em cada categoria (golfers e cores)
        if X in golfers and Y in golfers or \
           X in cores_calcas and Y in cores_calcas :
            restricoes[(X,Y)] = diferentes 
        
        if (X,Y) in restricoes :
            return restricoes[(X,Y)](a,b)

        
    return CSP(variaveis_golfers, dominios_golfers, vizinhos_golfers, restricoes_golfers)

Vamos agora criar um objecto `CSP` usando a função `CSP_golfers` que implementámos em cima:

In [9]:
p = CSP_golfers()

E verificar os valores das variáveis, dos domínios e dos vizinhos (grafo de restrições):

In [10]:
print("Variáveis = ", p.variables)

Variáveis =  ['Bob', 'Fred', 'Joe', 'Tom', 'Azuis', 'Laranja', 'Pretas', 'Vermelhas']


In [11]:
print("Domínios = ", p.domains)

Domínios =  {'Bob': [1, 2, 3, 4], 'Fred': [1, 2, 3, 4], 'Joe': [2], 'Tom': [1, 2, 3], 'Azuis': [1, 2, 3, 4], 'Laranja': [1, 2, 3, 4], 'Pretas': [1, 2, 3, 4], 'Vermelhas': [1, 2, 3, 4]}


In [12]:
print("Vizinhos = ", p.neighbors)

Vizinhos =  defaultdict(<class 'list'>, {'Bob': ['Fred', 'Joe', 'Tom', 'Pretas'], 'Fred': ['Bob', 'Joe', 'Tom', 'Azuis'], 'Joe': ['Bob', 'Fred', 'Tom'], 'Tom': ['Bob', 'Fred', 'Joe', 'Laranja'], 'Pretas': ['Bob', 'Azuis', 'Laranja', 'Vermelhas'], 'Azuis': ['Fred', 'Laranja', 'Pretas', 'Vermelhas'], 'Laranja': ['Tom', 'Azuis', 'Pretas', 'Vermelhas'], 'Vermelhas': ['Azuis', 'Laranja', 'Pretas']})


Verifique também que a função `restricoes_golfers` foi bem implementada, testando algumas afetações a pares de variáveis, confirmando que a função retorna `True`, se os valores não têm conflito com as restrições entre o par de variáveis passados como argumento para o par de valores concreto, e `False`, caso contrário.

In [13]:
# Restrição Fred = Azuis - 1
print(p.constraints('Fred', 2, "Azuis", 3))
print(p.constraints('Fred', 3, "Azuis", 2))

True
False


In [14]:
#Restrição Tom <> Laranja
print(p.constraints('Tom', 1, "Laranja", 1))
print(p.constraints('Tom', 1, "Azuis", 2))

False
None


In [15]:
#Restrição alldiff golfers
print(p.constraints('Bob', 1, "Fred", 2))
print(p.constraints('Bob', 1, "Fred", 1))   

True
False


In [16]:
#Restrição alldiff cores calças
print(p.constraints('Azuis', 1, "Laranja", 1))
print(p.constraints('Azuis', 1, "Laranja", 2))

False
True


Vamos então resolvê-lo invocando o algoritmo `backtrack_search()`. Este algoritmo será explicado em detalhe nas aulas da próxima semana. Para já considerem-no como uma caixa negra que recebe uma formulação do problema CSP e devolve uma solução (um *assignment* para cada uma das variáveis).

In [17]:
r = backtracking_search(p)
print('Afetação p = ',r)

Afetação p =  {'Bob': 4, 'Fred': 1, 'Joe': 2, 'Tom': 3, 'Azuis': 2, 'Laranja': 1, 'Pretas': 4, 'Vermelhas': 3}


#### Exercício 1
Formule usando a metodologia CSP, o problema de encontrar valores para as 3 variáveis A, B e C tais que A > B > C, quando o domínio é {1,2,3}
Teste-o usando a função 'backtracking_search()'.

In [26]:
# Resolução do exercício 1
def maior(x, y):
    return x > y

def menor(x, y):
    return x < y

def CSP_maiores ():
    """
    Retorna um objecto da classe CSP inicializado com as variáveis, os domínios,
    os vizinhos e restrições do problema dos maiores
    """

    #Definir Variáveis
    #letras
    variaveis_maiores = 'A B C'.split()

    # Definir Domínios
    # Devolve um dicionario com os domínios com as variáveis do problema dos Maiores.
    # É {1,2,3} para todas as variáveis
    dominios_maiores = {}
    for v in variaveis_maiores :
        dominios_maiores[v] = [1,2,3]

    #Definir Vizinhos
    #Cria o grafo de restrições com os arcos seguintes:
    #A: B C
    #B: C

    vizinhos_maiores = parse_neighbors('A: B C; B: C')

    #Definir função que verifica restrições binárias
    def restricoes_maiores(X, a, Y, b) :
        """
        A implementação em CSP assume que as restrições são binárias.
        Retorna True se (X=a,Y=b) satisfaz as restrições entre X e Y.

        Restrições unárias:

        Retrições Binárias:
        R_B1) A > B
        R_B2) B > C

        Restrições Globais:
        """

        # restrições binárias (necessário incluir (X,Y) e (Y,X))
        restricoes = { ('A','B'): maior, ('B','A'): menor,
                       ('B','C'): maior, ('C', 'B'): menor,
                       ('A', 'C'): maior, ('C', 'A'): menor }

        if (X,Y) in restricoes :
            return restricoes[(X,Y)](a,b)


    return CSP(variaveis_maiores, dominios_maiores, vizinhos_maiores, restricoes_maiores)

In [27]:
p = CSP_maiores()
print("Variáveis = ", p.variables)
print("Domínios = ", p.domains)
print("Vizinhos = ", p.neighbors)

r = backtracking_search(p)
print('Afetação p = ',r)

Variáveis =  ['A', 'B', 'C']
Domínios =  {'A': [1, 2, 3], 'B': [1, 2, 3], 'C': [1, 2, 3]}
Vizinhos =  defaultdict(<class 'list'>, {'A': ['B', 'C'], 'B': ['A', 'C'], 'C': ['A', 'B']})
Afetação p =  {'A': 3, 'B': 2, 'C': 1}


### Exercício 2 - Problema da coloração de mapas (Map Coloring)
<img src="figures/mapa_colorir.PNG" alt="Drawing" style="width: 200px;"/>

Considere o problema de colorir o mapa em cima usando 4 cores, por exemplo: R(ed), G(reen), B(lue) e Y(ellow). Defina um `CSP` inicializado com as variáveis, os domínios, os vizinhos e as restrições para este problema.

In [35]:
# Resolução do exercício 2
def CSP_mapa ():
    """
    Retorna um objecto da classe CSP inicializado com as variáveis, os domínios,
    os vizinhos e restrições do problema dos maiores
    """

    #Definir Variáveis
    # regiões
    variaveis_mapa = 'A B C D E'.split()

    # Definir Domínios
    # Devolve um dicionario com os domínios com as variáveis do problema de colorir o mapa.
    # É {'R', 'G', 'B', 'Y'} para todas as variáveis
    dominios_mapa = {}
    for v in variaveis_mapa :
        dominios_mapa[v] = 'R G B Y'.split()

    #Definir Vizinhos
    #Cria o grafo de restrições com os arcos seguintes:
    #A: B C D
    #B: C D
    #C: D
    #D: E

    vizinhos_mapa = parse_neighbors('A: B C D; B: C D; C: D; D: E')

    #Definir função que verifica restrições binárias
    def restricoes_mapa(X, a, Y, b) :
        """
        A implementação em CSP assume que as restrições são binárias.
        Retorna True se (X=a,Y=b) satisfaz as restrições entre X e Y.

        Restrições unárias:

        Retrições Binárias:
        R_B1) X != Y se Y é vizinho de X

        Restrições Globais:
        """
        if Y in vizinhos_mapa[X]:
            return diferentes(a, b)

        return True

    return CSP(variaveis_mapa, dominios_mapa, vizinhos_mapa, restricoes_mapa)

In [36]:
p = CSP_mapa()
print("Variáveis = ", p.variables)
print("Domínios = ", p.domains)
print("Vizinhos = ", p.neighbors)

r = backtracking_search(p)
print('Afetação p = ',r)

Variáveis =  ['A', 'B', 'C', 'D', 'E']
Domínios =  {'A': ['R', 'G', 'B', 'Y'], 'B': ['R', 'G', 'B', 'Y'], 'C': ['R', 'G', 'B', 'Y'], 'D': ['R', 'G', 'B', 'Y'], 'E': ['R', 'G', 'B', 'Y']}
Vizinhos =  defaultdict(<class 'list'>, {'A': ['B', 'C', 'D'], 'B': ['A', 'C', 'D'], 'C': ['A', 'B', 'D'], 'D': ['A', 'B', 'C', 'E'], 'E': ['D']})
Afetação p =  {'A': 'R', 'B': 'G', 'C': 'B', 'D': 'Y', 'E': 'R'}


### Exercício 3 - Problema genérico de coloração de mapa
Crie uma função que permite criar qualquer problema de coloração de mapas, fornecendo-lhe como input o conjunto de cores e a vizinhança. As cores definem o domínio de todas as variáveis e a partir da vizinhança obtêm-se as variáveis. Teste-o com o mapa do exercício anterior e com os seguintes países, dando-lhes as regiões adjacentes e as cores.

```python
australia = MapColoringCSP(list('RGBY'),
                           'SA: WA NT Q NSW V; NT: WA Q; NSW: Q V; T: ')

france = MapColoringCSP(list('RGBY'),
                        """AL: LO FC; AQ: MP LI PC; AU: LI CE BO RA LR MP; BO: CE IF CA FC RA
        AU; BR: NB PL; CA: IF PI LO FC BO; CE: PL NB NH IF BO AU LI PC; FC: BO
        CA LO AL RA; IF: NH PI CA BO CE; LI: PC CE AU MP AQ; LO: CA AL FC; LR:
        MP AU RA PA; MP: AQ LI AU LR; NB: NH CE PL BR; NH: PI IF CE NB; NO:
        PI; PA: LR RA; PC: PL CE LI AQ; PI: NH NO CA IF; PL: BR NB CE PC; RA:
        AU BO FC PA LR""")

usa = MapColoringCSP(list('RGBY'),
                     """WA: OR ID; OR: ID NV CA; CA: NV AZ; NV: ID UT AZ; ID: MT WY UT;
        UT: WY CO AZ; MT: ND SD WY; WY: SD NE CO; CO: NE KA OK NM; NM: OK TX AZ;
        ND: MN SD; SD: MN IA NE; NE: IA MO KA; KA: MO OK; OK: MO AR TX;
        TX: AR LA; MN: WI IA; IA: WI IL MO; MO: IL KY TN AR; AR: MS TN LA;
        LA: MS; WI: MI IL; IL: IN KY; IN: OH KY; MS: TN AL; AL: TN GA FL;
        MI: OH IN; OH: PA WV KY; KY: WV VA TN; TN: VA NC GA; GA: NC SC FL;
        PA: NY NJ DE MD WV; WV: MD VA; VA: MD DC NC; NC: SC; NY: VT MA CT NJ;
        NJ: DE; DE: MD; MD: DC; VT: NH MA; MA: NH RI CT; CT: RI; ME: NH;
        HI: ; AK: """)

```

In [38]:
# Resolução do exercício 3

# Resolução do exercício 2
def MapColoringCSP (cores, vizinhos):
    """
    Retorna um objecto da classe CSP inicializado com as variáveis, os domínios,
    os vizinhos e restrições do problema dos maiores
    """

    #Definir Vizinhos
    vizinhos_mapa = parse_neighbors(vizinhos)

    #Definir Variáveis
    # regiões
    variaveis_mapa = list(vizinhos_mapa.keys())

    # Definir Domínios
    # Devolve um dicionario com os domínios com as variáveis do problema de colorir o mapa.
    # É {'R', 'G', 'B', 'Y'} para todas as variáveis
    dominios_mapa = {}
    for v in variaveis_mapa :
        dominios_mapa[v] = cores

    #Definir função que verifica restrições binárias
    def restricoes_mapa(X, a, Y, b) :
        """
        A implementação em CSP assume que as restrições são binárias.
        Retorna True se (X=a,Y=b) satisfaz as restrições entre X e Y.

        Restrições unárias:

        Retrições Binárias:
        R_B1) X != Y se Y é vizinho de X

        Restrições Globais:
        """
        if Y in vizinhos_mapa[X]:
            return diferentes(a, b)

        return True

    return CSP(variaveis_mapa, dominios_mapa, vizinhos_mapa, restricoes_mapa)

In [39]:
australia = MapColoringCSP(list('RGBY'),
                           'SA: WA NT Q NSW V; NT: WA Q; NSW: Q V; T: ')

print("Variáveis = ", australia.variables)
print("Domínios = ", australia.domains)
print("Vizinhos = ", australia.neighbors)

r = backtracking_search(australia)
print('Afetação p = ',r)

Variáveis =  ['SA', 'WA', 'NT', 'Q', 'NSW', 'V']
Domínios =  {'SA': ['R', 'G', 'B', 'Y'], 'WA': ['R', 'G', 'B', 'Y'], 'NT': ['R', 'G', 'B', 'Y'], 'Q': ['R', 'G', 'B', 'Y'], 'NSW': ['R', 'G', 'B', 'Y'], 'V': ['R', 'G', 'B', 'Y']}
Vizinhos =  defaultdict(<class 'list'>, {'SA': ['WA', 'NT', 'Q', 'NSW', 'V'], 'WA': ['SA', 'NT'], 'NT': ['SA', 'WA', 'Q'], 'Q': ['SA', 'NT', 'NSW'], 'NSW': ['SA', 'Q', 'V'], 'V': ['SA', 'NSW']})
Afetação p =  {'SA': 'R', 'WA': 'G', 'NT': 'B', 'Q': 'G', 'NSW': 'B', 'V': 'G'}


In [40]:
france = MapColoringCSP(list('RGBY'),
                        """AL: LO FC; AQ: MP LI PC; AU: LI CE BO RA LR MP; BO: CE IF CA FC RA
        AU; BR: NB PL; CA: IF PI LO FC BO; CE: PL NB NH IF BO AU LI PC; FC: BO
        CA LO AL RA; IF: NH PI CA BO CE; LI: PC CE AU MP AQ; LO: CA AL FC; LR:
        MP AU RA PA; MP: AQ LI AU LR; NB: NH CE PL BR; NH: PI IF CE NB; NO:
        PI; PA: LR RA; PC: PL CE LI AQ; PI: NH NO CA IF; PL: BR NB CE PC; RA:
        AU BO FC PA LR""")

print("Variáveis = ", france.variables)
print("Domínios = ", france.domains)
print("Vizinhos = ", france.neighbors)

r = backtracking_search(france)
print('Afetação p = ',r)

Variáveis =  ['AL', 'LO', 'FC', 'AQ', 'MP', 'LI', 'PC', 'AU', 'CE', 'BO', 'RA', 'LR', 'IF', 'CA', 'BR', 'NB', 'PL', 'PI', 'NH', 'PA', 'NO']
Domínios =  {'AL': ['R', 'G', 'B', 'Y'], 'LO': ['R', 'G', 'B', 'Y'], 'FC': ['R', 'G', 'B', 'Y'], 'AQ': ['R', 'G', 'B', 'Y'], 'MP': ['R', 'G', 'B', 'Y'], 'LI': ['R', 'G', 'B', 'Y'], 'PC': ['R', 'G', 'B', 'Y'], 'AU': ['R', 'G', 'B', 'Y'], 'CE': ['R', 'G', 'B', 'Y'], 'BO': ['R', 'G', 'B', 'Y'], 'RA': ['R', 'G', 'B', 'Y'], 'LR': ['R', 'G', 'B', 'Y'], 'IF': ['R', 'G', 'B', 'Y'], 'CA': ['R', 'G', 'B', 'Y'], 'BR': ['R', 'G', 'B', 'Y'], 'NB': ['R', 'G', 'B', 'Y'], 'PL': ['R', 'G', 'B', 'Y'], 'PI': ['R', 'G', 'B', 'Y'], 'NH': ['R', 'G', 'B', 'Y'], 'PA': ['R', 'G', 'B', 'Y'], 'NO': ['R', 'G', 'B', 'Y']}
Vizinhos =  defaultdict(<class 'list'>, {'AL': ['LO', 'FC', 'FC', 'LO'], 'LO': ['AL', 'CA', 'FC', 'CA', 'AL', 'FC'], 'FC': ['AL', 'BO', 'CA', 'BO', 'CA', 'LO', 'AL', 'RA', 'LO', 'RA'], 'AQ': ['MP', 'LI', 'PC', 'LI', 'MP', 'PC'], 'MP': ['AQ', 'AU', 'LI', 'LR',

In [41]:
usa = MapColoringCSP(list('RGBY'),
                     """WA: OR ID; OR: ID NV CA; CA: NV AZ; NV: ID UT AZ; ID: MT WY UT;
        UT: WY CO AZ; MT: ND SD WY; WY: SD NE CO; CO: NE KA OK NM; NM: OK TX AZ;
        ND: MN SD; SD: MN IA NE; NE: IA MO KA; KA: MO OK; OK: MO AR TX;
        TX: AR LA; MN: WI IA; IA: WI IL MO; MO: IL KY TN AR; AR: MS TN LA;
        LA: MS; WI: MI IL; IL: IN KY; IN: OH KY; MS: TN AL; AL: TN GA FL;
        MI: OH IN; OH: PA WV KY; KY: WV VA TN; TN: VA NC GA; GA: NC SC FL;
        PA: NY NJ DE MD WV; WV: MD VA; VA: MD DC NC; NC: SC; NY: VT MA CT NJ;
        NJ: DE; DE: MD; MD: DC; VT: NH MA; MA: NH RI CT; CT: RI; ME: NH;
        HI: ; AK: """)

print("Variáveis = ", usa.variables)
print("Domínios = ", usa.domains)
print("Vizinhos = ", usa.neighbors)

r = backtracking_search(usa)
print('Afetação p = ',r)

Variáveis =  ['WA', 'OR', 'ID', 'NV', 'CA', 'AZ', 'UT', 'MT', 'WY', 'CO', 'ND', 'SD', 'NE', 'KA', 'OK', 'NM', 'TX', 'MN', 'IA', 'MO', 'AR', 'LA', 'WI', 'IL', 'KY', 'TN', 'MS', 'MI', 'IN', 'OH', 'AL', 'GA', 'FL', 'PA', 'WV', 'VA', 'NC', 'SC', 'NY', 'NJ', 'DE', 'MD', 'DC', 'VT', 'MA', 'CT', 'NH', 'RI', 'ME']
Domínios =  {'WA': ['R', 'G', 'B', 'Y'], 'OR': ['R', 'G', 'B', 'Y'], 'ID': ['R', 'G', 'B', 'Y'], 'NV': ['R', 'G', 'B', 'Y'], 'CA': ['R', 'G', 'B', 'Y'], 'AZ': ['R', 'G', 'B', 'Y'], 'UT': ['R', 'G', 'B', 'Y'], 'MT': ['R', 'G', 'B', 'Y'], 'WY': ['R', 'G', 'B', 'Y'], 'CO': ['R', 'G', 'B', 'Y'], 'ND': ['R', 'G', 'B', 'Y'], 'SD': ['R', 'G', 'B', 'Y'], 'NE': ['R', 'G', 'B', 'Y'], 'KA': ['R', 'G', 'B', 'Y'], 'OK': ['R', 'G', 'B', 'Y'], 'NM': ['R', 'G', 'B', 'Y'], 'TX': ['R', 'G', 'B', 'Y'], 'MN': ['R', 'G', 'B', 'Y'], 'IA': ['R', 'G', 'B', 'Y'], 'MO': ['R', 'G', 'B', 'Y'], 'AR': ['R', 'G', 'B', 'Y'], 'LA': ['R', 'G', 'B', 'Y'], 'WI': ['R', 'G', 'B', 'Y'], 'IL': ['R', 'G', 'B', 'Y'], 'KY': [

### Exercício 4 - Palavras Cruzadas pequeninas

Formule um problema de palavras cruzadas usando CSP, em que temos uma grelha 2x2 e 8 palavras possíveis para dispor 4 delas nas 2 linhas e 2 colunas. Cada palavra só pode aparecer numa das 4 regiões possíveis (2 linhas e 2 colunas) e as letras das palavras têm de ser as mesmas quando as palavras se intersectam. A figura apresenta uma solução para o problema com 8 palavras: {MA, OS, AS, TU, LI, RO, TO, MO}.
<img src="figures/PalavrasCruzadas2x2.png" alt="Drawing" style="width: 100px;"/>

In [ ]:
# Resolução das palavras-cruzadas pequeninas



### Exercício 5 - Escalonamento de tarefas

Formule o problema de escalonamento de tarefas de modo a determinar os tempos de início de cada tarefa que minimizam a duração total do processo. Verifique que está a funcionar bem invocando o ***backtracking_search()***.

<img src="figures/tarefas.PNG" alt="Drawing" style="width: 200px;"/>

In [ ]:
# Resolução do exercíco 5



### Exercício 6 - Problema das 4 rainhas

Considere o problema das 4 rainhas em que queremos dispor 4 rainhas num tabuleiro 4x4 tal que as raínhas não se ataquem entre si. As rainhas atacam ao longo das colunas, linhas e diagonais. Formule o problema CSP definindo as variáveis, os domínios, os vizinhos e as restrições para este problema.

<img src="figures/4queens.png" alt="Drawing" style="width: 150px;"/>

In [ ]:
# resolução do exercício 6



### Exercício 7 - Problema das N rainhas

Altere o problema das 4 rainhas que resolveu no exercício anterior e formule o problema das `N` rainhas. Defina a função **`CSP_N_rainhas(N)`** que recebe como parâmetro o número `N` de rainhas e devolve um `CSP` inicializado com as variáveis, os domínios, os vizinhos e as restrições para este problema. Confirme que está ok criando problemas com várias dimensões e invocando o método ***backtracking_search()***.

<img src="figures/16_queens_problem_trial.jpg" alt="Drawing" style="width: 200px;"/>

In [ ]:
## Resolução do exercício 7



### Exercício 8 - Problema do Sudoku
Formule como um CSP o problema do Sudoku para uma grelha 9x9.
Eis o exemplo de um problema em modo texto que pode usar para construir o CSP. Cosntrua uma função que recebe a string com o exemplo de puzzle e devolve a instância de CSP respetiva. Confirme que está ok invocando o método ***backtracking_search()*** para o exemplo a seguir.

```python
..3.2.6..
9..3.5..1
..18.64..
..81.29..
7.......8
..67.82..
..26.95..
8..2.3..9
..5.1.3..
```
temos a string de input:

```python
'..3.2.6..9..3.5..1..18.64....81.29..7.......8..67.82....26.95..8..2.3..9..5.1.3..'
```



In [ ]:
### Resolução do Exercício 8 - Problema do Sudoku



### Exercício 9 - Puzzle da Zebra

Considere o puzzle da Zebra https://en.wikipedia.org/wiki/Zebra_Puzzle, que consiste no seguinte puzzle lógico: Em 5 casas, cada uma com uma cor diferente, vivem 5 pessoas de nacionalidades diferentes, cada um prefere uma marca diferente de chocolates, uma bebida diferente, e um animal diferente. Dados os factos que se seguem, as perguntas a responder são: “Onde mora a zebra e em que casa bebem água “.
Formule o problema usando CSP e confirme que a solução é encontrada invocando o método ***backtracking_search()***.

<img src="figures/zebra.PNG" alt="Drawing" style="width: 600px;"/>


In [ ]:
# Resolução do Exer. 9

